# 04 — Compare Clustering Approaches & Validate Sub-Roles

Compare Path A (Twelve Football Qualities) vs Path B (Per-90 Z-Scores + PCA) clustering
results. Validate with bootstrap stability, UMAP visualization, and feature importance.

In [ ]:
import matplotlib
matplotlib.use("Agg")

import pandas as pd
import numpy as np
import json
import warnings
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, adjusted_rand_score

warnings.filterwarnings("ignore", category=FutureWarning)

# Dynamic path resolution
docs = Path("/Users/jorgepadilla/Documents")
for d in docs.iterdir():
    if "Jorge" in d.name and "MacBook" in d.name and d.is_dir():
        BASE = d / "thesis_data" / "raw_data"
        NB_DIR = d / "thesis_data" / "notebooks" / "player_subroles"
        break

# Load both clustering results
df_qual = pd.read_parquet(NB_DIR / "player_subroles_qualities.parquet")
df_zsc = pd.read_parquet(NB_DIR / "player_subroles_zscores.parquet")

# Load original player data (qualities version for features)
df_orig_qual = pd.read_parquet(NB_DIR / "player_data_qualities.parquet")
df_orig_zsc = pd.read_parquet(NB_DIR / "player_data_zscores.parquet")

# Load feature maps
with open(NB_DIR / "feature_maps.json", "r") as f:
    feature_maps = json.load(f)

position_quality_features = feature_maps["position_quality_features"]
position_zscore_features = feature_maps["position_zscore_features"]

positions = list(position_quality_features.keys())

print(f"Loaded Path A (Qualities): {df_qual.shape}")
print(f"Loaded Path B (Z-Scores):  {df_zsc.shape}")
print(f"Positions: {positions}")

## 1. Quantitative Comparison

In [ ]:
from sklearn.decomposition import PCA

# Re-compute silhouette scores for both paths
comparison_rows = []

for pos in positions:
    # --- Path A: Qualities ---
    feat_cols_a = position_quality_features[pos]
    pos_a = df_qual[df_qual["from_position"] == pos].dropna(subset=feat_cols_a).copy()
    X_a = pos_a[feat_cols_a].values
    scaler_a = StandardScaler()
    X_a_scaled = scaler_a.fit_transform(X_a)
    labels_a = pos_a["cluster_id"].values
    k_a = len(np.unique(labels_a))
    sil_a = silhouette_score(X_a_scaled, labels_a)
    method_a = pos_a["cluster_method"].iloc[0] if "cluster_method" in pos_a.columns else "Unknown"
    
    # --- Path B: Z-Scores + PCA ---
    feat_cols_b = position_zscore_features[pos]
    pos_b = df_zsc[df_zsc["from_position"] == pos].dropna(subset=feat_cols_b).copy()
    X_b = pos_b[feat_cols_b].values
    scaler_b = StandardScaler()
    X_b_scaled = scaler_b.fit_transform(X_b)
    # Apply PCA (90% variance)
    pca_b = PCA(random_state=42)
    pca_b.fit(X_b_scaled)
    cum_var = np.cumsum(pca_b.explained_variance_ratio_)
    n_comp = np.argmax(cum_var >= 0.90) + 1
    pca_b = PCA(n_components=n_comp, random_state=42)
    X_b_pca = pca_b.fit_transform(X_b_scaled)
    labels_b = pos_b["cluster_id"].values
    k_b = len(np.unique(labels_b))
    sil_b = silhouette_score(X_b_pca, labels_b)
    method_b = pos_b["cluster_method"].iloc[0] if "cluster_method" in pos_b.columns else "Unknown"
    
    comparison_rows.append({
        "Position": pos,
        "Path A k": k_a,
        "Path A Method": method_a,
        "Path A Sil": round(sil_a, 4),
        "Path B k": k_b,
        "Path B Method": method_b,
        "Path B Sil": round(sil_b, 4),
        "Better Path": "A" if sil_a > sil_b else "B",
    })

comp_df = pd.DataFrame(comparison_rows)
print("=" * 100)
print("Path A (Qualities) vs Path B (Z-Scores + PCA)")
print("=" * 100)
print(comp_df.to_string(index=False))

# Bar chart comparison
fig, ax = plt.subplots(figsize=(12, 6))
x = np.arange(len(positions))
width = 0.35

bars_a = ax.bar(x - width/2, comp_df["Path A Sil"], width, label="Path A (Qualities)",
                color="#4A6FA5", edgecolor="white", linewidth=0.5)
bars_b = ax.bar(x + width/2, comp_df["Path B Sil"], width, label="Path B (Z-Scores + PCA)",
                color="#E8724A", edgecolor="white", linewidth=0.5)

ax.set_xlabel("Position", fontsize=12)
ax.set_ylabel("Silhouette Score", fontsize=12)
ax.set_title("Silhouette Score Comparison: Qualities vs Z-Scores", fontsize=14, fontweight="bold")
ax.set_xticks(x)
ax.set_xticklabels(positions, rotation=20, ha="right", fontsize=10)
ax.legend(fontsize=11)
ax.grid(axis="y", alpha=0.3)

# Value labels
for bar in bars_a:
    ax.annotate(f"{bar.get_height():.3f}", xy=(bar.get_x() + bar.get_width()/2, bar.get_height()),
                xytext=(0, 4), textcoords="offset points", ha="center", fontsize=8)
for bar in bars_b:
    ax.annotate(f"{bar.get_height():.3f}", xy=(bar.get_x() + bar.get_width()/2, bar.get_height()),
                xytext=(0, 4), textcoords="offset points", ha="center", fontsize=8)

plt.tight_layout()
save_path = NB_DIR / "path_comparison_silhouette.png"
plt.savefig(save_path, dpi=150, bbox_inches="tight")
plt.show()
print(f"\nSaved: {save_path}")

## 2. Cluster Stability (Bootstrap)

In [ ]:
def bootstrap_stability(X, labels, best_k, method="KMeans", n_bootstrap=20, sample_frac=0.80, random_state=42):
    """
    Bootstrap stability: resample 80% of data, recluster, compute ARI vs full labels.
    Returns list of ARI values.
    """
    rng = np.random.RandomState(random_state)
    n = len(X)
    n_sample = int(n * sample_frac)
    ari_values = []
    
    for i in range(n_bootstrap):
        idx = rng.choice(n, size=n_sample, replace=False)
        X_boot = X[idx]
        labels_orig = labels[idx]
        
        if method == "KMeans":
            model = KMeans(n_clusters=best_k, n_init=10, random_state=rng.randint(10000))
        elif method == "Hierarchical":
            model = AgglomerativeClustering(n_clusters=best_k)
        else:  # GMM
            model = GaussianMixture(n_components=best_k, random_state=rng.randint(10000))
        
        boot_labels = model.fit_predict(X_boot)
        ari = adjusted_rand_score(labels_orig, boot_labels)
        ari_values.append(ari)
    
    return ari_values


# Run bootstrap for both paths
stability_rows = []

for pos in positions:
    print(f"\nBootstrapping {pos}...")
    
    # Path A
    feat_cols_a = position_quality_features[pos]
    pos_a = df_qual[df_qual["from_position"] == pos].dropna(subset=feat_cols_a).copy()
    X_a = StandardScaler().fit_transform(pos_a[feat_cols_a].values)
    labels_a = pos_a["cluster_id"].values
    k_a = len(np.unique(labels_a))
    method_a = pos_a["cluster_method"].iloc[0] if "cluster_method" in pos_a.columns else "KMeans"
    
    ari_a = bootstrap_stability(X_a, labels_a, k_a, method=method_a)
    
    # Path B (in PCA space)
    feat_cols_b = position_zscore_features[pos]
    pos_b = df_zsc[df_zsc["from_position"] == pos].dropna(subset=feat_cols_b).copy()
    X_b_raw = StandardScaler().fit_transform(pos_b[feat_cols_b].values)
    pca_tmp = PCA(random_state=42)
    pca_tmp.fit(X_b_raw)
    cum_var = np.cumsum(pca_tmp.explained_variance_ratio_)
    n_comp = np.argmax(cum_var >= 0.90) + 1
    X_b = PCA(n_components=n_comp, random_state=42).fit_transform(X_b_raw)
    labels_b = pos_b["cluster_id"].values
    k_b = len(np.unique(labels_b))
    method_b = pos_b["cluster_method"].iloc[0] if "cluster_method" in pos_b.columns else "KMeans"
    
    ari_b = bootstrap_stability(X_b, labels_b, k_b, method=method_b)
    
    stability_rows.append({
        "Position": pos,
        "Path A ARI (mean)": round(np.mean(ari_a), 4),
        "Path A ARI (std)": round(np.std(ari_a), 4),
        "Path B ARI (mean)": round(np.mean(ari_b), 4),
        "Path B ARI (std)": round(np.std(ari_b), 4),
        "More Stable": "A" if np.mean(ari_a) > np.mean(ari_b) else "B",
    })
    
    print(f"  Path A: ARI={np.mean(ari_a):.4f} +/- {np.std(ari_a):.4f}")
    print(f"  Path B: ARI={np.mean(ari_b):.4f} +/- {np.std(ari_b):.4f}")

stab_df = pd.DataFrame(stability_rows)
print("\n" + "=" * 85)
print("Bootstrap Cluster Stability (20 iterations, 80% sample)")
print("=" * 85)
print(stab_df.to_string(index=False))

## 3. UMAP Visualization

In [ ]:
try:
    import umap
    HAS_UMAP = True
except ImportError:
    try:
        import umap.umap_ as umap
        HAS_UMAP = True
    except ImportError:
        HAS_UMAP = False
        print("UMAP not installed. Falling back to t-SNE for visualization.")

if not HAS_UMAP:
    from sklearn.manifold import TSNE

# Choose the best path for each position based on silhouette comparison
best_path = {}
for _, row in comp_df.iterrows():
    best_path[row["Position"]] = row["Better Path"]

colors_map = ["#4A6FA5", "#E8724A", "#6B9F6B", "#9B6FA5", "#C4A44A", "#5AAFAF", "#D46A6A", "#8B8B8B"]

fig, axes = plt.subplots(2, 3, figsize=(18, 11))
fig.suptitle("2D Projection of Player Sub-Roles (Best Path per Position)", fontsize=14, fontweight="bold")

for idx, pos in enumerate(positions):
    ax = axes[idx // 3, idx % 3]
    path = best_path[pos]
    
    if path == "A":
        feat_cols = position_quality_features[pos]
        pos_df = df_qual[df_qual["from_position"] == pos].dropna(subset=feat_cols).copy()
        X = StandardScaler().fit_transform(pos_df[feat_cols].values)
        labels = pos_df["cluster_id"].values
        path_label = "Qualities"
    else:
        feat_cols = position_zscore_features[pos]
        pos_df = df_zsc[df_zsc["from_position"] == pos].dropna(subset=feat_cols).copy()
        X_raw = StandardScaler().fit_transform(pos_df[feat_cols].values)
        pca_tmp = PCA(random_state=42)
        pca_tmp.fit(X_raw)
        cum_var = np.cumsum(pca_tmp.explained_variance_ratio_)
        n_comp = np.argmax(cum_var >= 0.90) + 1
        X = PCA(n_components=n_comp, random_state=42).fit_transform(X_raw)
        labels = pos_df["cluster_id"].values
        path_label = "Z-Scores+PCA"
    
    # 2D projection
    if HAS_UMAP:
        reducer = umap.UMAP(n_components=2, random_state=42, n_neighbors=30, min_dist=0.3)
        embedding = reducer.fit_transform(X)
        method_name = "UMAP"
    else:
        # Use a subsample for t-SNE if data is large
        max_tsne = 10000
        if len(X) > max_tsne:
            rng = np.random.RandomState(42)
            sample_idx = rng.choice(len(X), size=max_tsne, replace=False)
            X_sub = X[sample_idx]
            labels_sub = labels[sample_idx]
        else:
            X_sub = X
            labels_sub = labels
        reducer = TSNE(n_components=2, random_state=42, perplexity=30)
        embedding = reducer.fit_transform(X_sub)
        labels = labels_sub
        method_name = "t-SNE"
    
    # Scatter plot
    unique_labels = np.unique(labels)
    for i, lab in enumerate(unique_labels):
        mask = labels == lab
        ax.scatter(embedding[mask, 0], embedding[mask, 1], c=colors_map[i % len(colors_map)],
                   label=f"Cluster {lab}", alpha=0.5, s=8, edgecolors="none")
    
    ax.set_title(f"{pos} (Path {path}: {path_label})", fontsize=11)
    ax.set_xlabel(f"{method_name} 1", fontsize=9)
    ax.set_ylabel(f"{method_name} 2", fontsize=9)
    ax.legend(fontsize=8, markerscale=2)
    ax.grid(True, alpha=0.2)

plt.tight_layout()
save_path = NB_DIR / "umap_subroles.png"
plt.savefig(save_path, dpi=150, bbox_inches="tight")
plt.show()
print(f"Saved: {save_path}")

## 4. Feature Importance (XGBoost / Random Forest)

In [ ]:
# Try XGBoost, fall back to RandomForest
try:
    from xgboost import XGBClassifier
    USE_XGB = True
    print("Using XGBoost for feature importance.")
except ImportError:
    USE_XGB = False
    print("XGBoost not installed. Using RandomForest for feature importance.")
    from sklearn.ensemble import RandomForestClassifier


# Use quality features (Path A) for interpretability
fig, axes = plt.subplots(2, 3, figsize=(18, 11))
fig.suptitle("Feature Importance for Cluster Prediction (Quality Features)", fontsize=14, fontweight="bold")

importance_results = {}

for idx, pos in enumerate(positions):
    ax = axes[idx // 3, idx % 3]
    feat_cols = position_quality_features[pos]
    pos_df = df_qual[df_qual["from_position"] == pos].dropna(subset=feat_cols).copy()
    X = pos_df[feat_cols].values
    y = pos_df["cluster_id"].values
    
    if len(np.unique(y)) < 2:
        ax.set_title(f"{pos} (only 1 cluster, skipped)")
        continue
    
    try:
        if USE_XGB:
            model = XGBClassifier(
                n_estimators=100, max_depth=4, random_state=42,
                eval_metric="mlogloss", verbosity=0,
                use_label_encoder=False
            )
        else:
            model = RandomForestClassifier(n_estimators=100, max_depth=6, random_state=42)
        
        model.fit(X, y)
        importances = model.feature_importances_
        
        # Sort by importance
        sorted_idx = np.argsort(importances)[::-1]
        top_n = min(10, len(feat_cols))
        top_idx = sorted_idx[:top_n]
        
        short_names = [c.replace("from_", "") for c in feat_cols]
        top_names = [short_names[i] for i in top_idx]
        top_imps = [importances[i] for i in top_idx]
        
        importance_results[pos] = list(zip(top_names, top_imps))
        
        # Horizontal bar chart
        y_pos = np.arange(top_n)
        ax.barh(y_pos, top_imps[::-1], color="#4A6FA5", edgecolor="white", linewidth=0.5)
        ax.set_yticks(y_pos)
        ax.set_yticklabels(top_names[::-1], fontsize=9)
        ax.set_xlabel("Importance", fontsize=9)
        ax.set_title(f"{pos} (k={len(np.unique(y))})", fontsize=11)
        ax.grid(axis="x", alpha=0.3)
        
    except Exception as e:
        ax.set_title(f"{pos} (error: {str(e)[:40]})")
        print(f"Error for {pos}: {e}")

plt.tight_layout()
save_path = NB_DIR / "feature_importance.png"
plt.savefig(save_path, dpi=150, bbox_inches="tight")
plt.show()
print(f"Saved: {save_path}")

## 5. Recommendation

In [ ]:
print("=" * 90)
print("FINAL RECOMMENDATION PER POSITION")
print("=" * 90)

final_rows = []

for i, pos in enumerate(positions):
    row_comp = comp_df.iloc[i]
    row_stab = stab_df.iloc[i]
    
    # Decision logic: prefer higher silhouette; if close, prefer higher stability
    sil_a = row_comp["Path A Sil"]
    sil_b = row_comp["Path B Sil"]
    ari_a = row_stab["Path A ARI (mean)"]
    ari_b = row_stab["Path B ARI (mean)"]
    
    # If silhouette difference is small (<0.02), use stability as tiebreaker
    if abs(sil_a - sil_b) < 0.02:
        recommended_path = "A" if ari_a >= ari_b else "B"
        reason = "stability tiebreaker"
    else:
        recommended_path = "A" if sil_a > sil_b else "B"
        reason = "higher silhouette"
    
    if recommended_path == "A":
        rec_k = row_comp["Path A k"]
        rec_method = row_comp["Path A Method"]
        rec_sil = sil_a
        rec_ari = ari_a
        src_df = df_qual
    else:
        rec_k = row_comp["Path B k"]
        rec_method = row_comp["Path B Method"]
        rec_sil = sil_b
        rec_ari = ari_b
        src_df = df_zsc
    
    # Get cluster names and sizes
    pos_data = src_df[src_df["from_position"] == pos]
    cluster_info = pos_data.groupby(["cluster_id", "cluster_name"]).size().reset_index(name="count")
    
    print(f"\n--- {pos} ---")
    print(f"  Recommended: Path {recommended_path} ({reason})")
    print(f"  Method: {rec_method}, k={rec_k}")
    print(f"  Silhouette: {rec_sil:.4f}, Bootstrap ARI: {rec_ari:.4f}")
    print(f"  Sub-roles:")
    for _, cr in cluster_info.iterrows():
        print(f"    [{cr['cluster_id']}] {cr['cluster_name']:40s} n={cr['count']:,}")
    
    for _, cr in cluster_info.iterrows():
        final_rows.append({
            "position": pos,
            "recommended_path": recommended_path,
            "method": rec_method,
            "k": rec_k,
            "silhouette": rec_sil,
            "bootstrap_ari": rec_ari,
            "cluster_id": cr["cluster_id"],
            "cluster_name": cr["cluster_name"],
            "n_players": cr["count"],
        })

# Build final combined parquet
# For each position, take data from the recommended path
final_dfs = []
for pos in positions:
    recommended_path = best_path[pos]
    # Re-check with stability tiebreaker
    row_comp_i = comp_df[comp_df["Position"] == pos].iloc[0]
    row_stab_i = stab_df[stab_df["Position"] == pos].iloc[0]
    sil_a = row_comp_i["Path A Sil"]
    sil_b = row_comp_i["Path B Sil"]
    ari_a = row_stab_i["Path A ARI (mean)"]
    ari_b = row_stab_i["Path B ARI (mean)"]
    if abs(sil_a - sil_b) < 0.02:
        rec_path = "A" if ari_a >= ari_b else "B"
    else:
        rec_path = "A" if sil_a > sil_b else "B"
    
    if rec_path == "A":
        pos_data = df_qual[df_qual["from_position"] == pos].copy()
    else:
        pos_data = df_zsc[df_zsc["from_position"] == pos].copy()
    
    pos_data["recommended_path"] = rec_path
    final_dfs.append(pos_data)

df_final = pd.concat(final_dfs, ignore_index=True)

# Keep essential columns
essential = ["wy_player_id", "from_position", "from_team_id", "from_season", "from_Minutes",
             "cluster_id", "cluster_name", "cluster_method", "recommended_path"]
# Add player name if present
for nc in ["from_player_name", "from_short_name", "player_name"]:
    if nc in df_final.columns:
        essential.insert(1, nc)
        break

available = [c for c in essential if c in df_final.columns]
df_final_out = df_final[available].copy()

save_path = NB_DIR / "player_subroles_final.parquet"
df_final_out.to_parquet(save_path, index=False)
print(f"\n{'='*90}")
print(f"Final output saved: {save_path}")
print(f"Shape: {df_final_out.shape}")
print(f"\nSummary:")
summary = df_final_out.groupby(["from_position", "cluster_name"]).size().reset_index(name="count")
print(summary.to_string(index=False))